<h1><center>The Battle of the Neighborhoods - Boston</center></h1>

<h1><center>Applied Data Science Capstone Project</center></h1>

<h4>About this Project</h4>
In this project, we will evaluate the Boston area neighborhoods in order to determine the most advantagous areas to live based on a customer's preferences.  

<h4>Discription and background of the problem</h4>
A potential job opening is available in the greater Boston area and a family is considering the best location to search for future homes.  The family has never been to Boston and unfamiliar with the various neighborhoods.  The family has an opportunity to house hunt during a four-day weekend and would like to search neighborhoods that offer the best opportunities as places with live with services and businesses that they frequent.  

<h4>Data and how it will solve the problem</h4>
The Foursquare.com data provides location information on different types of venues by category and name.  The family would like to search neighborhoods based on the proximity to favorite venues.

The family has a number of store preferences and would like to exam the types of venues in Foursquare.com data.  They would like to search specific names of businesses and determine if they can identify the locations with the best range to their preferred 